#### Q1.非线性优化转化为线性规划

In [2]:
# 导入模块
import gurobipy as grb
import numpy as np
import pandas as pd

##### 计算线性规划模型各参数

In [143]:
#定义模型参数类
class Parameters:
    def __init__(self,k,sigma,cost,reinsurance,
                 freq,rate,Resource,MaxPay,payrate,
                 alpha,P0,require,Base,period=1):
        self.k=k
        self.sigma=sigma
        self.cost=cost
        self.reinsurance=reinsurance
        self.freq=freq
        self.rate=rate
        self.period=period
        self.Resource=Resource
        self.MaxPay=MaxPay
        self.payrate=payrate
        self.alpha=alpha
        self.P0=P0
        self.require=require
        self.Base=Base
    def calculate_parameters(self):
        
        τ=self.k*np.exp(self.sigma)
        # 目标函数常数项
        # C=lnτ-ln(cost)-ln(reinsurance)-ln(10*freq)-ln(exp(-rate*period))
        C=np.log(τ)-np.log(self.cost)-np.log(self.reinsurance)
        -np.log(10*self.freq)-np.log(np.exp(-self.rate*self.period))
        # 约束条件常数项1：Pmax=ln(MaxPay)
        Pmax=self.MaxPay
        # 约束条件常数项2：Rmax=ln(Resource)
        Rmax=self.Resource
        # 约束条件常数项3：PAYRATEmin=ln(payrate)
        PAYRATEmin=self.payrate
        # 约束条件常数项4： alpha=ln(alpha)
        alpha=self.alpha
        # alpha=np.log(self.alpha)
        # 约束条件常数项5：p0=lnP0
        p0=self.P0
        # Require=lnRequire
        Require=self.require
        # freq=lnfreq
        freq=self.freq
        Base=self.Base
        sigma=np.exp(-self.rate*self.period)
        return τ,C,Pmax,Rmax,PAYRATEmin,alpha,p0,Require,freq,Base,sigma

    def model(self):
        τ,C,Pmax,Rmax,PAYRATEmin,alpha,p0,Require,freq,Base,sigma=self.calculate_parameters()
        m=grb.Model()
        # #定义变量

        x=m.addVar(name="x",lb=0)
        y=m.addVar(name="y",lb=0)
        z=m.addVar(name="z",lb=PAYRATEmin,ub=1)
        pi=m.addVar(name="pi")
        # 添加约束
        # m.addConstr(x<=Pmax,name='c1')
        # m.addConstr(x*y<=Rmax,name='c2')
        m.addConstr(z<=1,name='c3')
        m.addConstr(z>=PAYRATEmin,name='c4')
        # m.addConstr(2*sigma*x-y+z==-6,name='c5')
        m.addConstr(x<=p0*z*self.freq,name='c5')
        m.addConstr(pi==τ*(1-self.cost)*(1-self.reinsurance)*x*y-10000*y*self.freq*sigma*z,name='c6')
        m.addConstr(y<=Require,name='c7')
        m.addConstr(y*z*τ*freq<=1000,name='c8')
        m.addConstr(x*y>=freq*p0*z,name='c9')

        m.setObjective(pi,grb.GRB.MAXIMIZE)
        m.optimize()
        if m.status == grb.GRB.OPTIMAL:
            m.printAttr('X')
        # 对结果取指数
        x_value = x.X
        y_value = y.X
        z_value = z.X
        pi_value =pi.X
        print('最优解的目标函数值为：',pi_value)
        print(f"Solution values: x={x_value}, y={y_value}, z={z_value}")
        return x_value,y_value,z_value,pi_value
        
        
        





#### 模型参数实例化

In [149]:
param=Parameters(k=0.03,sigma=0,cost=0.061,
                    reinsurance=0.07,freq=0.07,rate=0.0482,
                    Resource=0,MaxPay=0,
                    payrate=0.01,alpha=-0.05,period=1,
                    Base=10000000,require=382500,P0=1000000)
list={}
list=param.calculate_parameters()
print(list)
param.model()

(0.03, 1.949583554421622, 0, 0, 0.01, -0.05, 1000000, 382500, 0.07, 10000000, 0.9529431793814942)
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 9 7945HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 4 rows, 4 columns and 5 nonzeros
Model fingerprint: 0x92feb3e3
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 7e+04]
  QMatrix range    [2e-03, 7e+02]
  QLMatrix range   [1e+00, 7e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+00]
  RHS range        [1e-02, 4e+05]
  QRHS range       [1e+03, 1e+03]
Presolve removed 3 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 6 columns, 35 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 6 continuous,

(70000.0, 382500.0, 1.0, 446303591.2206049)

In [9]:


m=grb.Model()

#定义变量
x=m.addVar(name="x",lb=0)
y=m.addVar(name="y",lb=0)
z=m.addVar(name="z",lb=-0.69)
pi=m.addVar(name="pi")

# 添加约束
m.addConstr(x<=60000,name='c1')
m.addConstr(x+y<=70000,name='c2')
m.addConstr(z<=0,name='c3')
m.addConstr(z>=-0.69,name='c4')
m.addConstr(2*x-2*y-z==500,name='c5')
m.addConstr(x-2*y-z-pi==0,name='c6')

m.setObjective(pi,grb.GRB.MAXIMIZE)
m.optimize()

print('最优解的目标函数值为：',m.objVal)
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")
# 对结果取指数
x_value = np.exp(x.X)
y_value = np.exp(y.X)
z_value = np.exp(z.X)
pi_value = np.exp(pi.X)
print(f"Solution values: x={x_value}, y={y_value}, z={z_value}, pi={pi_value}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 9 7945HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0xc892a2e9
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 7e-01]
  RHS range        [7e-01, 7e+04]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5034500e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.503450000e+02
最优解的目标函数值为： 250.345
Solution values: x=249.655, y=0.0, z=-0.69
Solution values: x=2.6533154113341567e+108, y=1.0, z=0.5015760690660556, pi=5.289956150170154e+108
